### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1005.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

### Solution

In [85]:
from datetime import datetime, timezone, timedelta
from functools import namedtuple
import numbers
import itertools

In [86]:
class TimeZone():
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone can not be empty!')
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Houre offset must be integral!')
            
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minutes offset must be integral!')
            
        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError('Minutes offset must be between -59 and 59!')
            
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and 14:00')
            
        self._name = name
        self._offset = offset
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        
    @property    
    def offset(self):
        'Return offset TimeZone'
        return self._offset
    
    @property
    def name(self):
        'Return name TimeZone'
        return self._name

In [87]:
class Account():
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5
    
    ParseResult = namedtuple('ParseResult',
                             'account_number transaction_code transaction_id time time_utc')
    
    def __init__(self, account_num, 
                 first_name, 
                 last_name,
                 time_zone=TimeZone('UTC', 0, 0),
                 start_balance=0):
        'This is constructor for Account'
        
        self._account_num = account_num
        self.validate_set_name('_first_name', first_name, 'First name')
        self.validate_set_name('_last_name', last_name, 'Last name')
        
        if not isinstance(time_zone, TimeZone):
            raise ValueError('Preferred timezone must be instance TimeZone!')
        self._time_zone = time_zone
        
        self._balance = 0 if start_balance < 0 else start_balance
        
    def validate_set_name(self, name_property, value, title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{title} can't be empty!")
        setattr(self, name_property, str(value))
        
    @property
    def account_number(self):
        'Return account number'
        return self._account_num
        
    @property    
    def balance(self):
        'Return balance'
        return self._balance
    
    @property
    def first_name(self):
        'Return first name'
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        'Set first name'
        self.validate_set_name('_first_name', value, 'First name')
        
    @property
    def last_name(self):
        'Return last name'
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        'Set last name'
        self.validate_set_name('_last_name', value, 'Last name')
    
    @property
    def full_name(self):
        'Return full name'
        return self.first_name + ' ' + self.last_name
    
    @classmethod
    def get_interest_rate(cls):
        'Return interest rate'
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        'Set interest rate'
        if value is None or not isinstance(value, numbers.Real):
            raise ValueError("Interest rate can be real number!")
        cls._interest_rate = value
    
    def deposit(self, dep):
        'Function deposit'
        if dep is None or dep < 0:
            raise ValueError('Deposit must be positive!')
            
        transaction_id = next(Account.transaction_counter)
        tmp_time = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        self._balance += dep
        return f'D-{self._account_num}-{tmp_time}-{transaction_id}'
        
    def pay_interest(self):
        'Function pay interest'
        transaction_id = next(Account.transaction_counter)
        tmp_time = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        self._balance += self._balance * self.get_interest_rate() * (10 ** -2)
        return f'I-{self._account_num}-{tmp_time}-{transaction_id}'
        
    def withdraw(self, money):
        'Function withdraw'
        transaction_id = next(Account.transaction_counter)
        tmp_time = datetime.utcnow().strftime('%Y%m%d%H%M%S')
        if self._balance < money:
            return f'X-{self._account_num}-{tmp_time}-{transaction_id}'
        else:
            self._balance -= money
            return f'W-{self._account_num}-{tmp_time}-{transaction_id}'
    
    @staticmethod
    def parse_code(s, tmz=TimeZone('UTC', 0, 0)):
        'Parser confirmation code transaction'
        parts = s.split('-')
        if len(parts) < 4:
            raise ValueError('Invalid code')
        transaction_code, account_number, time_str, transaction_id = parts
        
        try:
            time_utc = datetime.strptime(time_str, '%Y%m%d%H%M%S')
        except ValueError as ex:
            raise ValueError('Wrong datetime') from ex
        
        time_preferred = time_utc + tmz.offset
        time_preferred_str =f"{time_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({tmz.name})"
        
        return ParseResult(account_number=account_number,
                           transaction_code=transaction_code,
                           transaction_id=int(transaction_id),
                           time=time_preferred_str,
                           time_utc=time_utc.isoformat()
                          )
        
    
    def __repr__(self):
        'Representation Account'
        return 'Account(account_num={0}, first_name={1}, last_name={2}, timezone={3}, balance={4})'.format(self.account_num,
                                                                                                          self.first_name,
                                                                                                          self.last_name,
                                                                                                          self.time_zone,
                                                                                                          self._balance)   

In [88]:
a = Account('A100', 'Eric', 'Idle', time_zone=TimeZone('MST', -7, 0), start_balance=100)
print(a.balance)
print(a.deposit(150.02))
print(a.balance)
print(a.withdraw(0.02))
print(a.balance)
Account.set_interest_rate(1.0)
print(a.get_interest_rate())
print(a.pay_interest())
print(a.balance)
print(a.withdraw(1000))

100
D-A100-20200710133322-100
250.02
W-A100-20200710133322-101
250.0
1.0
I-A100-20200710133322-102
252.5
X-A100-20200710133322-103


### Test

In [89]:
import unittest

In [90]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [91]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        self.account_number = 'A100'
        self.first_name = 'First'
        self.last_name = 'Last'
        self.tz = TimeZone('TZ', 1, 30)
        self.balance = 100.0
    
    def test_create_account(self):
        a = Account(self.account_number, self.first_name, self.last_name,
                    self.tz, self.balance)
        
        self.assertEqual(self.account_number, a.account_number)
        self.assertEqual(self.first_name, a.first_name)
        self.assertEqual(self.last_name, a.last_name)
        self.assertEqual(self.first_name + ' ' + self.last_name, a.full_name)
        self.assertEqual(self.balance, a.balance)
        
    def test_create_account_blank_first_name(self):
        self.first_name = ''
        
        with self.assertRaises(ValueError):
            a = Account(self.account_number, self.first_name, self.last_name)
            
        self.first_name = None
        
        with self.assertRaises(ValueError):
            a = Account(self.account_number, self.first_name, self.last_name)
            
    def test_account_withdraw(self):
        withdrawal_amount = 20
        
        a = Account(self.account_number, self.first_name, self.last_name,
                    self.tz, self.balance)
        
        conf_code = a.withdraw(withdrawal_amount)
        self.assertTrue(conf_code.startswith('W-'))
        self.assertEqual(self.balance - withdrawal_amount, a.balance)
        last_save_balance = a.balance
        
        withdrawal_amount = 1000
        conf_code = a.withdraw(withdrawal_amount)
        self.assertTrue(conf_code.startswith('X-'))
        self.assertEqual(last_save_balance, a.balance)
        
    def test_account_deposit(self):
        a = Account(self.account_number, self.first_name, self.last_name,
                    self.tz, self.balance)
        
        deposit_amounts = (20, 150, 500, 300)
        balances = (
            100.0 + 20.0,
            120.0 + 150.0,
            270.0 + 500.0,
            770.0 + 300.0
        )
        
        for dep_amount, tmp_balance in zip(deposit_amounts, balances):
            a.deposit(dep_amount)
            self.assertEqual(a.balance, tmp_balance)
            
    def test_neg_deposit(self):
        a = Account(self.account_number, self.first_name, self.last_name,
                    self.tz, self.balance)
        
        dep_amount = -50
        with self.assertRaises(ValueError):
            a.deposit(dep_amount)
        
        

In [92]:
run_tests(TestAccount)

test_account_deposit (__main__.TestAccount) ... ok
test_account_withdraw (__main__.TestAccount) ... ok
test_create_account (__main__.TestAccount) ... ok
test_create_account_blank_first_name (__main__.TestAccount) ... ok
test_neg_deposit (__main__.TestAccount) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK
